In [ ]:
import time, os
from utils.generic import load_environment_variables
from utils.generate_data import generate_nyctaxistream_async
from utils.kafka import write_to_kafka_async,list_kafka_topics,delete_kafka_topic,create_kafka_topic,close_kafka_admin
from utils.spark import start_spark_session,read_from_kafka,read_spark_parquet,write_spark_stream
import asyncio
from delta.tables import DeltaTable
from dotenv import load_dotenv

# load env vars
config = load_environment_variables()

topicname = "nyctaxistream"
kafka_bootstrap_servers = os.getenv("KAFKA_BOOTSTRAP_SERVERS")
delta_path = os.getenv("DELTA_PATH_NYCTAXI")
write_folder_path = os.getenv("WRITE_FOLDER_PATH_NYCTAXI")
schema = os.getenv("SCHEMA_NYCTAXI")
merge_keys = os.getenv("MERGE_KEYS_NYCTAXI").split(",")
raw_file_path = os.path.join(os.getenv("RAW_PATH_NYCTAXI"),os.getenv("RAW_FILE_NAME"))


In [ ]:
topics = list_kafka_topics()
print(topics)

if len(topics) > 0:
    print("Resetting Kafka topics...\n")
    for topic in topics:
        print(f"Deleting topic {topic}")
        delete_kafka_topic(topic)

    #Wait and poll until topics disappear
    print("Waiting for topics to be fully deleted...")
    for _ in range(10):  # up to ~10 seconds
        time.sleep(1)
        remaining = list_kafka_topics()
        if not any(t in topics for t in remaining):
            break
    else:
        print("Topics still pending deletion, continuing anyway...")

print("\nCreating Kafka topic...")  
create_kafka_topic(topicname)
close_kafka_admin()
print("\nKafka topics reset complete.")


In [ ]:


number_of_events = 20
desired_latency = 1

B03494_stream = write_to_kafka_async(topicname, generate_nyctaxistream_async(number_of_events, desired_latency,dispatching_base_num=["B03494"]))
B03496_stream =  write_to_kafka_async(topicname, generate_nyctaxistream_async(number_of_events, desired_latency,dispatching_base_num=["B03496"]))

await asyncio.gather(B03494_stream, B03496_stream) #sequential execution of both streams